# Mem0 Workflow Mastery Notebook

Use this notebook to practice the full set of Mem0-powered memory operations implemented in this project.

## Goals
- Confirm your environment can initialize Mem0
- Walk through the add, search, get all, update, history, and reset workflows
- Capture reusable helpers and patterns for future experiments

## Prerequisites
1. `mem0_enabled=true` in your `.env` (see `MEM0_IMPLEMENTATION_SUMMARY.md`)
2. API keys for the configured LLM and embedder available as environment variables
3. Dependencies installed (`uv sync`)
4. ChromaDB data directory available at `config.chroma_db_path` (created automatically if missing)

---
## 1. Environment Setup

In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
src_path = project_root / 'src'
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

from dotenv import load_dotenv
load_dotenv()

print(f'Project root: {project_root}')
print(f'Source path added: {src_path}')

Project root: /Users/kevinknox/coding/acc-llamaindex
Source path added: /Users/kevinknox/coding/acc-llamaindex/src


---
## 2. Inspect Mem0 Configuration

In [2]:
from acc_llamaindex.config import config

print('Mem0 feature flag :', config.mem0_enabled)
print('Vector store      :', config.mem0_vector_store)
print('LLM provider      :', config.mem0_llm_provider)
print('LLM model         :', config.mem0_llm_model)
print('Embedder model    :', config.mem0_embedder_model)
print('History DB path   :', config.mem0_history_db_path)
print('Chroma DB path    :', config.chroma_db_path)

if not config.mem0_enabled:
    print('WARNING: Mem0 is disabled. Update your environment and restart the kernel before running workflows.')

Mem0 feature flag : True
Vector store      : chroma
LLM provider      : openai
LLM model         : gpt-5-nano-2025-08-07
Embedder model    : text-embedding-3-small
History DB path   : /Users/kevinknox/coding/acc-llamaindex/data/mem0_history.db
Chroma DB path    : /Users/kevinknox/coding/acc-llamaindex/data/chroma_db


---
## 3. Initialize Mem0 Client

In [3]:
from acc_llamaindex.application.memory_service.mem0_client import mem0_client

mem0_available = mem0_client.is_enabled()
print('Mem0 client ready:', mem0_available)

if not mem0_available:
    print('Mem0 did not initialize. Confirm API keys are set and mem0_enabled=true before proceeding.')

2025-10-24 18:22:13.495 | INFO     | acc_llamaindex.application.memory_service.mem0_client:__init__:62 - Mem0 client initialized (provider=openai, model=gpt-5-nano-2025-08-07)


Mem0 client ready: True


---
## 4. Helper Utilities
These helpers keep state across the workflow steps and make it easy to inspect responses.

In [4]:
import json
from datetime import datetime
from typing import Any, Dict

def pretty_print(label: str, payload: Any) -> None:
    """Utility to pretty-print Mem0 responses."""
    print(f'\n=== {label} ===')
    if payload is None:
        print('None')
        return
    if isinstance(payload, (dict, list)):
        print(json.dumps(payload, indent=2, default=str))
    else:
        print(payload)

def make_run_ids(prefix: str = 'mem0-notebook') -> Dict[str, str]:
    timestamp = datetime.utcnow().strftime('%Y%m%d%H%M%S')
    return {
        'user_id': f'{prefix}-user',
        'session_id': f'{prefix}-session-{timestamp}',
        'agent_id': f'{prefix}-agent'
    }

mem0_ids: Dict[str, str] = {}
last_memory_id: str | None = None

if mem0_available:
    mem0_ids = make_run_ids()
    pretty_print('Identifiers', mem0_ids)
else:
    print('Skipping helper initialization because Mem0 is unavailable.')


=== Identifiers ===
{
  "user_id": "mem0-notebook-user",
  "session_id": "mem0-notebook-session-20251024222215",
  "agent_id": "mem0-notebook-agent"
}


---
## 5. Workflow: Add Conversation to Memory

In [5]:
if mem0_available:
    messages = [
        {
            'role': 'user',
            'content': 'Can you remind me which Incoterms we used for the Lagos textiles shipment?'
        },
        {
            'role': 'assistant',
            'content': 'We agreed on FOB Lagos for that shipment.'
        }
    ]
    add_result = mem0_client.add(
        messages=messages,
        user_id=mem0_ids['user_id'],
        agent_id=mem0_ids['agent_id'],
        session_id=mem0_ids['session_id'],
        metadata={'channel': 'notebook-demo'}
    )
    pretty_print('Add Result', add_result)
    last_memory_id = add_result.get('memory', {}).get('id') or add_result.get('id')
    print('Stored memory id:', last_memory_id)
else:
    print('Mem0 unavailable; skip add workflow.')

2025-10-24 18:22:34.342 | INFO     | acc_llamaindex.application.memory_service.mem0_client:add:108 - Added memory for user=mem0-notebook-user, run=mem0-notebook-session-20251024222215



=== Add Result ===
{
  "results": [
    {
      "id": "f59a2928-fae8-4453-8262-32ee1e3f1838",
      "memory": "Incoterms used: FOB Lagos",
      "event": "ADD"
    }
  ]
}
Stored memory id: None


---
## 6. Workflow: Search Memories

In [ ]:
if mem0_available:
    search_results = mem0_client.search(
        query='FOB Lagos shipment reminder',
        user_id=mem0_ids['user_id'],
        session_id=mem0_ids['session_id'],
        limit=5 
    )
    # Extract 'results' array then get the first result
    results_list = search_results.get('results', [])
    pretty_print('Search Results', search_results)
    if results_list:
        last_memory_id = results_list[0].get('id') 
        print('Memory id from search:', last_memory_id)
else:
    print('Mem0 unavailable; skip search workflow.')

2025-10-24 18:26:36.698 | INFO     | acc_llamaindex.application.memory_service.mem0_client:search:148 - Found 1 memories for query='FOB Lagos shipment reminder...', user=mem0-notebook-user, session=mem0-notebook-session-20251024222215



=== Search Results ===
{
  "results": [
    {
      "id": "f59a2928-fae8-4453-8262-32ee1e3f1838",
      "memory": "Incoterms used: FOB Lagos",
      "hash": "4144db44a5d292fab283bf5fe3cd2c66",
      "metadata": {
        "channel": "notebook-demo"
      },
      "score": 0.4477316737174988,
      "created_at": "2025-10-24T15:22:34.337177-07:00",
      "updated_at": null,
      "user_id": "mem0-notebook-user",
      "agent_id": "mem0-notebook-agent",
      "run_id": "mem0-notebook-session-20251024222215"
    }
  ]
}
Memory id from search: f59a2928-fae8-4453-8262-32ee1e3f1838


---
## 7. Workflow: Get All Memories

In [14]:
if mem0_available:
    all_memories = mem0_client.get_all(
        user_id=mem0_ids['user_id'],
        session_id=mem0_ids['session_id']
    )
    pretty_print('All Memories', all_memories)
    if all_memories and not last_memory_id:
        last_memory_id = all_memories[0].get('id')
        print('Memory id from get_all:', last_memory_id)
else:
    print('Mem0 unavailable; skip get_all workflow.')

2025-10-24 20:38:03.656 | INFO     | acc_llamaindex.application.memory_service.mem0_client:get_all:185 - Retrieved 1 memories for user=mem0-notebook-user, agent=None, session=mem0-notebook-session-20251024222215



=== All Memories ===
{
  "results": [
    {
      "id": "f59a2928-fae8-4453-8262-32ee1e3f1838",
      "memory": "Customer prefers FOB Lagos for textiles shipments; remind proactively.",
      "hash": "b261da6cd8bbf2da5867f8598b9dd25f",
      "metadata": {
        "channel": "notebook-demo"
      },
      "created_at": "2025-10-24T15:22:34.337177-07:00",
      "updated_at": "2025-10-24T16:12:31.767721-07:00",
      "user_id": "mem0-notebook-user",
      "agent_id": "mem0-notebook-agent",
      "run_id": "mem0-notebook-session-20251024222215"
    }
  ]
}


---
## 8. Workflow: Update a Memory

In [13]:
if mem0_available and last_memory_id:
    update_payload = 'Customer prefers FOB Lagos for textiles shipments; remind proactively.'
    update_result = mem0_client.update(last_memory_id, update_payload)
    pretty_print('Update Result', update_result)
else:
    print('Skipping update; either Mem0 unavailable or no memory id captured.')

2025-10-24 19:12:31.777 | INFO     | acc_llamaindex.application.memory_service.mem0_client:update:235 - Updated memory f59a2928-fae8-4453-8262-32ee1e3f1838



=== Update Result ===
{
  "message": "Memory updated successfully!"
}


---
## 9. Workflow: Inspect Memory History

In [15]:
if mem0_available and last_memory_id:
    history_result = mem0_client.history(last_memory_id)
    pretty_print('History', history_result)
else:
    print('Skipping history; either Mem0 unavailable or no memory id captured.')

2025-10-24 20:42:03.922 | INFO     | acc_llamaindex.application.memory_service.mem0_client:history:258 - Retrieved history for memory f59a2928-fae8-4453-8262-32ee1e3f1838



=== History ===
[
  {
    "id": "14392461-9376-48a9-9adc-f4b0c0ccb9b5",
    "memory_id": "f59a2928-fae8-4453-8262-32ee1e3f1838",
    "old_memory": null,
    "new_memory": "Incoterms used: FOB Lagos",
    "event": "ADD",
    "created_at": "2025-10-24T15:22:34.337177-07:00",
    "updated_at": null,
    "is_deleted": false,
    "actor_id": null,
    "role": null
  },
  {
    "id": "1e3fcc9a-6451-459f-96d7-07f8b6eaed0c",
    "memory_id": "f59a2928-fae8-4453-8262-32ee1e3f1838",
    "old_memory": "Incoterms used: FOB Lagos",
    "new_memory": "Customer prefers FOB Lagos for textiles shipments; remind proactively.",
    "event": "UPDATE",
    "created_at": "2025-10-24T15:22:34.337177-07:00",
    "updated_at": "2025-10-24T16:12:31.767721-07:00",
    "is_deleted": false,
    "actor_id": null,
    "role": null
  }
]


---
## 10. Workflow: Reset Memories (Optional)
Be cautious with resets in shared environments. Use filters to target only the identifiers created in this notebook.

In [ ]:
if mem0_available:
    should_reset = False  # Flip to True when you want to clear the notebook data
    if should_reset:
        reset_result = mem0_client.reset(
            user_id=mem0_ids['user_id'],
            session_id=mem0_ids['session_id']
        )
        pretty_print('Reset Executed', reset_result)
    else:
        print('Reset skipped. Set should_reset=True to clear notebook data.')
else:
    print('Mem0 unavailable; skip reset workflow.')

---
## 11. API Routes Reference
The FastAPI routes in `src/acc_llamaindex/infrastructure/api/routes/memory_routes.py` expose the same Mem0 operations.
Use them to validate behavior outside this notebook (e.g., with `curl` or Postman).

```bash
# Health check
curl http://localhost:8000/memory/health

# Search memories
curl -X POST http://localhost:8000/memory/search \
  -H 'Content-Type: application/json' \
  -d '{"query": "FOB Lagos", "user_id": "mem0-notebook-user"}'
```

---
## 12. Next Steps
1. Run the notebook end-to-end with Mem0 enabled to seed working data.
2. Experiment with additional metadata filters and search queries.
3. Integrate the workflows into automated tests or eval pipelines as needed.